# Converting raw .txt files from labview to fretbursts compatible HDF5 files

This notebook will convert the .txt files written by our labview acquisition software into HDF5 files which can be read into other FRETbursts notebooks. 

It is set up to convert the many txt files created within a folder for a single run into one single coherent HDF5. If you want to convert multiple runs quickly, use the batch converter, which iterates this process through a folder *of* folders

In [2]:
import phconvert as phc
import csv
phc.__version__
import numpy as np
import os

Start by naming the folder your files are in, this code will then find all of the files in it, and report which it has found.

In [3]:
folder = 'itertest/' #put the RAW filename here
savename = 'itertest/itertest1.hdf5' #put the save location / name herefilenames


filenames = [] #LEAVE THIS EMPTY!!!
for file in os.listdir(folder):        
    if file.endswith("T.txt"):
        datafile = file
        filenames.append(folder+file)
    if file.endswith("etadata.txt"):
        metafile = (os.path.join(folder, file))
    else:
        continue
print("Found data files:")
print(filenames)
print("Found metadata:")
print(metafile)

Found data files:
['itertest/000_Sample_TXT.txt', 'itertest/001_Sample_TXT.txt', 'itertest/002_Sample_TXT.txt', 'itertest/003_Sample_TXT.txt', 'itertest/004_Sample_TXT.txt', 'itertest/005_Sample_TXT.txt', 'itertest/006_Sample_TXT.txt', 'itertest/007_Sample_TXT.txt', 'itertest/008_Sample_TXT.txt', 'itertest/009_Sample_TXT.txt']
Found metadata:
itertest/Metadata.txt


Read in the columns from each file and concatenate them. It will report the first and last photons from every file read in. If they do not logically follow on from each other then something has gone wrong.

In [4]:
detectors = np.empty([0], dtype=int)
timestamps = np.empty([0], dtype=int)
for file in filenames:
    with open(file) as inf:
        reader = csv.reader(inf, delimiter="	")
        ftimestamps = list(zip(*reader))[0]    
    with open(file) as inf:
        reader = csv.reader(inf, delimiter="	")
        fdetectors = list(zip(*reader))[1]
    fdetectors = np.asarray(fdetectors)
    ftimestamps = np.asarray(ftimestamps)
    ftimestamps = np.int64(ftimestamps)
    fdetectors = np.uint8(fdetectors)
    print(ftimestamps)
    timestamps = np.concatenate([timestamps, ftimestamps])
    detectors = np.concatenate([detectors, fdetectors]) 
timestamps = np.int64(timestamps)
detectors = np.uint8(detectors)
print("done")

[   123714    160225    161123 ... 599983745 599984206 599915692]
[ 599993950  600032171  600052430 ... 1199971316 1199981047 1199982137]
[1200010541 1200050996 1200051844 ... 1799935683 1799951790 1799954167]
[1799981344 1799982141 1799990848 ... 2399962206 2399971148 2399908865]
[2400000750 2400032669 2400054177 ... 2999932846 2999934343 2999895810]
[2999994166 3000002753 3000023043 ... 3599950542 3599876107 3599911627]
[3599992196 3600001367 3600012784 ... 4199881395 4199892624 4199925411]
[4199963829 4199967127 4199980948 ... 4799930674 4799944358 4799942531]
[4799961233 4799970159 4799982500 ... 5399903952 5399921280 5399939415]
[5399954271 5399973116 5399981243 ... 6000253252 6000253902 6000301714]
done


Now we can create an HDF5 file by combining the data with the metadata

In [6]:
with open(metafile) as inf:
            reader = csv.reader(inf, delimiter=":")
            metaread = list(zip(*reader))
meta = [x[1:] for x in metaread[1]]
D_ON, D_OFF, A_ON, A_OFF = int(meta[10]), int(meta[11]), int(meta[12]), int(meta[13])
sample_name = meta[5]
buffer_name = meta[8]
dye_names = meta[6] + "," + meta[7]
timestamps_unit = 10e-9  # 10 ns, units are always S.I.
photon_data = dict(
    timestamps=timestamps,
    detectors=detectors,
    timestamps_specs={'timestamps_unit': timestamps_unit})
setup = dict(
    num_pixels = 2,                   # using 2 detectors
    num_spots = 1,                    # a single confoca excitation
    num_spectral_ch = 2,              # donor and acceptor detection 
    num_polarization_ch = 1,          # no polarization selection 
    num_split_ch = 1,                 # no beam splitter
    modulated_excitation = False,     # CW excitation, no modulation 
    excitation_alternated = [True],  # CW excitation, no modulation 
    lifetime = False,                 # no TCSPC in detection    
    excitation_wavelengths = [515e-9, 635e-9],         # List of excitation wavelenghts
    excitation_cw = [True],                    # List of booleans, True if wavelength is CW
    detection_wavelengths = [580e-9, 690e-9],  # Nominal center wavelength 
)
provenance = dict( 
    software='Custom Labview Software - Craggs Lab',
    filename=filenames[0])
identity = dict(
    author= meta[1][3],
    author_affiliation="University of Sheffield") #CHANGE THIS WHERE NECESSARY
measurement_specs = dict(
    measurement_type = 'smFRET-usALEX',
    detectors_specs = {'spectral_ch1': [0],  # list of donor's detector IDs
                       'spectral_ch2': [1]},  # list of acceptor's detector IDs,
    alex_period = 10000,
    alex_offset = 0,
    alex_excitation_period1 = (D_ON, D_OFF),
    alex_excitation_period2 = (A_ON, A_OFF)
    )
photon_data['measurement_specs'] = measurement_specs
data = dict(
    description=meta[9],
    photon_data = photon_data,
    setup=setup,
    identity=identity,
    provenance=provenance
)
phc.hdf5.save_photon_hdf5(data, h5_fname=savename, overwrite=True)

Saving: itertest/itertest1.hdf5
